In [ ]:
#Connect to DB via MongoS (our mongos is running at port 27200 on localhost)

In [5]:
import pymongo
cli = pymongo.MongoClient('localhost', 27200)
db = cli.mongoMart
coll = db.restaurants

# B-tree indexes

In [7]:
coll.drop_indexes()

In [8]:
coll.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)]}}

In [6]:
coll.find_one()

{'_id': ObjectId('60abc72b99e7e0f50266f015'),
 'name': 'Daniel',
 'cuisine': 'Sushi',
 'stars': 1.8,
 'address': {'street': '844 Ozaiti Terrace',
  'city': 'Puugilu',
  'state': 'DC',
  'zipcode': '87625'}}

In [9]:
coll.create_index('name')

'name_1'

In [ ]:
coll.create_index([('name', 1)])

In [10]:
coll.create_index([('name', pymongo.ASCENDING)])

'name_1'

In [13]:
q = coll.find({'name': 'Dunkin Donuts'})
q

In [14]:
set(doc['cuisine'] for doc in q)

set()

In [15]:
coll.create_index([('name', 1), ('cuisine', 1)])

'name_1_cuisine_1'

In [16]:
coll.drop_index('name_1')

In [17]:
q = coll.find(
    {'name': 'Dunkin Donuts', 'cuisine': 'Armenian'},
    {'name': 1, 'borough': 1, '_id': 0})
list(q)    

[]

# Using `explain()`

In [18]:
coll.drop_indexes()

In [19]:
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan

{'queryPlanner': {'mongosPlannerVersion': 1,
  'winningPlan': {'stage': 'SHARD_MERGE',
   'shards': [{'shardName': 'shard3rs',
     'connectionString': 'shard3rs/localhost:2007,localhost:2008,localhost:2009',
     'serverInfo': {'host': 'DESKTOP-P7RG4I0',
      'port': 2009,
      'version': '4.4.6',
      'gitVersion': '72e66213c2c3eab37d9358d5e78ad7f5c1d0d0d7'},
     'plannerVersion': 1,
     'namespace': 'mongoMart.restaurants',
     'indexFilterSet': False,
     'parsedQuery': {'name': {'$eq': 'Dunkin Donuts'}},
     'winningPlan': {'stage': 'SHARDING_FILTER',
      'inputStage': {'stage': 'COLLSCAN',
       'filter': {'name': {'$eq': 'Dunkin Donuts'}},
       'direction': 'forward'}},
     'rejectedPlans': []},
    {'shardName': 'shard2rs',
     'connectionString': 'shard2rs/localhost:2004,localhost:2005,localhost:2006',
     'serverInfo': {'host': 'DESKTOP-P7RG4I0',
      'port': 2004,
      'version': '4.4.6',
      'gitVersion': '72e66213c2c3eab37d9358d5e78ad7f5c1d0d0d7'},
    

In [20]:
plan['queryPlanner']['winningPlan']

{'stage': 'SHARD_MERGE',
 'shards': [{'shardName': 'shard3rs',
   'connectionString': 'shard3rs/localhost:2007,localhost:2008,localhost:2009',
   'serverInfo': {'host': 'DESKTOP-P7RG4I0',
    'port': 2009,
    'version': '4.4.6',
    'gitVersion': '72e66213c2c3eab37d9358d5e78ad7f5c1d0d0d7'},
   'plannerVersion': 1,
   'namespace': 'mongoMart.restaurants',
   'indexFilterSet': False,
   'parsedQuery': {'name': {'$eq': 'Dunkin Donuts'}},
   'winningPlan': {'stage': 'SHARDING_FILTER',
    'inputStage': {'stage': 'COLLSCAN',
     'filter': {'name': {'$eq': 'Dunkin Donuts'}},
     'direction': 'forward'}},
   'rejectedPlans': []},
  {'shardName': 'shard2rs',
   'connectionString': 'shard2rs/localhost:2004,localhost:2005,localhost:2006',
   'serverInfo': {'host': 'DESKTOP-P7RG4I0',
    'port': 2004,
    'version': '4.4.6',
    'gitVersion': '72e66213c2c3eab37d9358d5e78ad7f5c1d0d0d7'},
   'plannerVersion': 1,
   'namespace': 'mongoMart.restaurants',
   'indexFilterSet': False,
   'parsedQuery

In [ ]:
plan['executionStats']['executionStages']

In [ ]:
from pprint import pprint
def plan_summary(plan):
    winningPlan = plan['queryPlanner']['winningPlan']
    executionStages = plan['executionStats']['executionStages']
    print('Winning Plan')
    pprint(winningPlan)
    print()
    print('Execution Stages')
    pprint(executionStages)

In [ ]:
plan_summary(plan)

In [ ]:
coll.create_index('name')

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan_summary(plan)

In [ ]:
plan['executionStats']['executionStages']

In [ ]:
from pprint import pprint
def plan_summary(plan):
    winningPlan = plan['queryPlanner']['winningPlan']
    stages = []
    stage = plan['executionStats']['executionStages']
    while stage:
        inputStage = stage.pop('inputStage', None)
        stages.append(stage)
        stage = inputStage
    stages.reverse()
    print('Winning Plan')
    pprint(winningPlan)
    print()
    print('Execution Stages')
    for stage in stages:
        pprint(stage)
        print('--')

In [ ]:
plan_summary(plan)

In [ ]:
q = coll.find({'name': 'Dunkin Donuts', 'cuisine': 'Armenian'})
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.create_index([('name', 1), ('cuisine', 1)])
coll.drop_index('name_1')

In [ ]:
q = coll.find({'name': 'Dunkin Donuts', 'cuisine': 'Armenian'})
plan = q.explain()
plan_summary(plan)

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
plan = q.explain()
plan_summary(plan)

### Range / prefix queries

In [ ]:
import re
q = coll.find({'name': re.compile('^Dunkin.* Donuts$')})
plan = q.explain()
plan_summary(plan)

In [ ]:
q.distinct('name')

In [ ]:
coll.drop_indexes()
coll.create_index([('cuisine', 1), ('name', 1)])

In [ ]:
q = coll.find({'name': re.compile('^Dunkin.* Donuts$')})
plan = q.explain()
plan_summary(plan)

In [ ]:
q = coll.find({'name': re.compile('^Dunkin.* Donuts$'), 'cuisine': 'Donuts'})
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.find_one()

In [ ]:
coll.drop_indexes()
coll.create_index('name')

In [ ]:
q = coll.find({
    'name': 'Dunkin Donuts', 
    'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.create_index([('name',1), ('address.zipcode', 1)])

In [ ]:
q = coll.find({'name': 'Dunkin Donuts', 'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
plan = q.explain()
plan_summary(plan)

# Sorting

In [ ]:
coll.drop_indexes()
coll.create_index('name')
q = coll.find({'name': 'Dunkin Donuts'})
q = q.sort('address.zipcode')
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.drop_indexes()
coll.create_index([('name', 1), ('address.zipcode', 1)])

In [ ]:
q = coll.find({'name': 'Dunkin Donuts'})
q = q.sort('address.zipcode')
plan = q.explain()
plan_summary(plan)

In [ ]:
q = coll.find({
    'name': 'Dunkin Donuts', 
    'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
q = q.sort('borough')
plan = q.explain()
plan_summary(plan)

In [ ]:
coll.drop_indexes()
coll.create_index([
    ('name', 1),
    ('borough', 1),
    ('address.zipcode', 1)
])
q = coll.find({
    'name': 'Dunkin Donuts', 
    'address.zipcode': {'$gte': '10460', '$lte': '10462'}})
q = q.sort('borough')
plan = q.explain()
plan_summary(plan)

## Sorting with Ranges Trade-off

- Sort before range examines more _index keys_
- Range before sort generates more _stages_ in the query

Always arrange you index with

1. Exact matches
1. Sort keys
1. Range matches

... in that order